# Crop simulation

Doing imports 

In [ ]:
%matplotlib inline
#%matplotlib notebook
import sys
import pcse
import pandas as pd
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
print("This notebook was built with:")
print(f"python version: {sys.version}")
print(f"PCSE version: {pcse.__version__}")

This notebook was built with:
python version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
PCSE version: 6.0.9


Import the crop, site, soil and agro data

In [ ]:
from pcse.util import DummySoilDataProvider
from pcse.input import WOFOST72SiteDataProvider 
import yaml
from pcse.base import ParameterProvider

#load crop file
with open('data/crop/potato.yaml', 'r') as file: #alternative: from pcse.input import YAMLCropDataProvider
    yaml_file = yaml.safe_load(file)

#flatten crop file: remove the nested structure
def flatten_nested_dict(d):
    flat = {}
    for k, v in d.items():
        if isinstance(v, dict):
            flat.update(flatten_nested_dict(v))  # flatten recursively
        else:
            flat[k] = v
    return flat

cropd = {}
for subdict in yaml_file["CropParameters"].values():
    if isinstance(subdict, dict):
        cropd.update(flatten_nested_dict(subdict))

#export to .yaml file (only for debugging purpose)
with open("flattened_crop.yaml", "w") as f_out:
    yaml.dump(cropd, f_out, default_flow_style=False)

#load dummy soil file
soild = DummySoilDataProvider() #use dummy soil: https://pcse.readthedocs.io/en/stable/code.html#pcse.util.DummySoilDataProvider

#load some standard site, WAV = initial soil moisture
sited = WOFOST72SiteDataProvider(WAV=10) #https://pcse.readthedocs.io/en/stable/code.html#wofost72sitedataprovider

#put everything together with ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

#agro file (is not completed yet)
with open('data/agro/potato_agro_test.yaml', 'r') as file:
    agromanagement = yaml.safe_load(file)

running the model with the given files

In [ ]:
#running the model
from pcse.models import Wofost72_WLP_CWB, Wofost72_PP
wofsim = Wofost72_WLP_CWB(parameters, weatherdataprovider, agromanagement)

wofsim.run(days=10)
df_results = pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()